In [51]:
import pandas as pd

df = pd.read_csv("melb_data.csv")

df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [52]:
print(df.isnull().sum())

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64


In [53]:
df['Car'] = df['Car'].fillna(df['Car'].mode()[0])
df['CouncilArea'] = df['CouncilArea'].fillna(df['CouncilArea'].mode()[0])

# 'BuildingArea' and 'YearBuilt' are numerical -> Fill with Median
df['BuildingArea'] = df['BuildingArea'].fillna(df['BuildingArea'].median())
df['YearBuilt'] = df['YearBuilt'].fillna(df['YearBuilt'].median())



In [49]:
print(df.isnull().sum())

Suburb           0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
BuildingArea     0
YearBuilt        0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64


In [55]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import LabelEncoder

X = df.drop(["Price", "Date" ,"Address"], axis=1)
y = df["Price"]
label_encoders = {}

categorical_cols = df.select_dtypes(include="object").columns
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le


mi_scores = mutual_info_regression(X, y,random_state=42)
mi_scores = pd.Series(mi_scores, name="MI Score", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print(mi_scores)

Postcode         0.323818
Suburb           0.230776
Rooms            0.212965
Distance         0.209373
Propertycount    0.204996
Bedroom2         0.204419
BuildingArea     0.190274
CouncilArea      0.179196
Type             0.174965
Longtitude       0.154187
Lattitude        0.147816
Landsize         0.141268
Bathroom         0.127056
SellerG          0.125576
Method           0.114210
Regionname       0.106031
Car              0.088473
YearBuilt        0.082458
Name: MI Score, dtype: float64


In [56]:
correlation_room_bedroom2 = df[['Rooms', 'Bedroom2']].corr()

correlation_room_bathroom = df[['Rooms', 'Bathroom']].corr()

print(correlation_room_bedroom2)
print(correlation_room_bathroom)


            Rooms  Bedroom2
Rooms     1.00000   0.94419
Bedroom2  0.94419   1.00000
             Rooms  Bathroom
Rooms     1.000000  0.592934
Bathroom  0.592934  1.000000


In [57]:
cols_to_drop = ['Bedroom2', 'YearBuilt','Address', 'Date', 'Postcode', 'Lattitude', 'Longtitude']
df = df.drop(cols_to_drop, axis=1, errors='ignore')

# Check which columns remain
print(df.columns)


Index(['Suburb', 'Rooms', 'Type', 'Price', 'Method', 'SellerG', 'Distance',
       'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'CouncilArea',
       'Regionname', 'Propertycount'],
      dtype='object')


In [58]:
df.head()


,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Bathroom,Car,Landsize,BuildingArea,CouncilArea,Regionname,Propertycount
0,0,2,0,1480000.0,1,23,2.5,1.0,1.0,202.0,126.0,31,2,4019.0
1,0,2,0,1035000.0,1,23,2.5,1.0,0.0,156.0,79.0,31,2,4019.0
2,0,3,0,1465000.0,3,23,2.5,2.0,0.0,134.0,150.0,31,2,4019.0
3,0,3,0,850000.0,0,23,2.5,2.0,1.0,94.0,126.0,31,2,4019.0
4,0,4,0,1600000.0,4,155,2.5,1.0,2.0,120.0,142.0,31,2,4019.0


In [59]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import LabelEncoder

X = df.drop(["Price"], axis=1)
y = df["Price"]
y = df["Price"]

mi_scores = mutual_info_regression(X, y,random_state=42)
mi_scores = pd.Series(mi_scores, name="MI Score", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print(mi_scores)

Suburb           0.230606
Rooms            0.221413
Distance         0.208670
Propertycount    0.206390
BuildingArea     0.195594
CouncilArea      0.186556
Type             0.172935
Landsize         0.147822
SellerG          0.127026
Bathroom         0.121655
Method           0.112866
Regionname       0.099765
Car              0.089127
Name: MI Score, dtype: float64


In [60]:
unique_count = df['Suburb'].nunique()
print(f"Total unique suburbs: {unique_count}")

unique_sellerG = df['SellerG'].nunique()
print(f"Total unique sellers: {unique_sellerG}")

# 2. See the top 10 most frequent suburbs to understand the distribution
print("\nTop 10 most frequent suburbs:")
print(df['Suburb'].value_counts().head(10))

print("\nTop 10 most frequent Sellers:")
print(df['SellerG'].value_counts().head(10))

Total unique suburbs: 314
Total unique sellers: 268

Top 10 most frequent suburbs:
Suburb
236    359
237    260
30     249
234    239
46     222
118    220
259    202
136    195
147    191
80     190
Name: count, dtype: int64

Top 10 most frequent Sellers:
SellerG
155    1565
106    1316
260    1167
16     1011
196     701
135     659
32      632
23      393
28      342
73      301
Name: count, dtype: int64


In [61]:
import json
suburb_means = df.groupby('Suburb')['Price'].mean()
df['Suburb_Encoded'] = df['Suburb'].map(suburb_means)

seller_means = df.groupby('SellerG')['Price'].mean()
df['SellerG_Encoded'] = df['SellerG'].map(seller_means)

suburb_dict = suburb_means.to_dict()
seller_dict = seller_means.to_dict()

with open('suburb_map.json', 'w') as f:
    json.dump(suburb_dict, f)

with open('seller_map.json', 'w') as f:
    json.dump(seller_dict, f)

df = df.drop(['Suburb', 'SellerG'], axis=1)

print(df[['Suburb_Encoded', 'SellerG_Encoded', 'Price']].head())

   Suburb_Encoded  SellerG_Encoded      Price
0    1.060366e+06     1.018522e+06  1480000.0
1    1.060366e+06     1.018522e+06  1035000.0
2    1.060366e+06     1.018522e+06  1465000.0
3    1.060366e+06     1.018522e+06   850000.0
4    1.060366e+06     1.014976e+06  1600000.0


In [64]:
print(df.head())

   Rooms  Type      Price  Method  Distance  Bathroom  Car  Landsize  \
0      2     0  1480000.0       1       2.5       1.0  1.0     202.0   
1      2     0  1035000.0       1       2.5       1.0  0.0     156.0   
2      3     0  1465000.0       3       2.5       2.0  0.0     134.0   
3      3     0   850000.0       0       2.5       2.0  1.0      94.0   
4      4     0  1600000.0       4       2.5       1.0  2.0     120.0   

   BuildingArea  CouncilArea  Regionname  Propertycount  Suburb_Encoded  \
0         126.0           31           2         4019.0    1.060366e+06   
1          79.0           31           2         4019.0    1.060366e+06   
2         150.0           31           2         4019.0    1.060366e+06   
3         126.0           31           2         4019.0    1.060366e+06   
4         142.0           31           2         4019.0    1.060366e+06   

   SellerG_Encoded  
0     1.018522e+06  
1     1.018522e+06  
2     1.018522e+06  
3     1.018522e+06  
4     1.014

In [37]:

df.to_csv('processed_melbourne_housing.csv', index=False)

In [69]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder

# 1. Load the raw data
df = pd.read_csv('melb_data.csv')

# --- GROUP A: TARGET ENCODING (Map to Price) ---
# (Matches your 'Suburb_Encoded' and 'SellerG_Encoded' columns)
suburb_map = df.groupby('Suburb')['Price'].mean().to_dict()
seller_map = df.groupby('SellerG')['Price'].mean().to_dict()

def get_label_map(column_name):
    le = LabelEncoder()
    le.fit(df[column_name].astype(str))
    return dict(zip(le.classes_, range(len(le.classes_))))

type_map = get_label_map('Type')
method_map = get_label_map('Method')
council_map = get_label_map('CouncilArea')
region_map = get_label_map('Regionname')

prop_map = df.groupby('Suburb')['Propertycount'].first().to_dict()

# 2. Save ALL Maps
files = {
    "suburb_map.json": suburb_map,
    "seller_map.json": seller_map,
    "type_map.json": type_map,
    "method_map.json": method_map,
    "council_map.json": council_map,
    "region_map.json": region_map,
    "property_map.json": prop_map
}

for filename, data in files.items():
    with open(filename, 'w') as f:
        json.dump(data, f)

print("Hybrid Maps Generated! Move these 7 files to your backend folder.")


 Hybrid Maps Generated! Move these 7 files to your backend folder.
